<a href="https://colab.research.google.com/github/respect5716/deep-learning-paper-implementation/blob/main/02_Vision/VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG

## 0. Paper

### Info
* Title:  Very Deep Convolutional Networks for Large-Scale Image Recognition
* Author: Karen Simonyan
* Task: Image Classification
* URL: https://arxiv.org/abs/1409.1556v6


### Features
* Dataset: CIFAR-10
* Batch Normalization
* Average Pooling

### Reference
* https://github.com/kuangliu/pytorch-cifar


## 1. Setting

In [1]:
!pip install -q wandb
!pip install -q transformers
!pip install -q pytorch_lightning

In [2]:
import os
import wandb
from glob import glob
from typing import Optional

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import pytorch_lightning as pl

_ = pl.utilities.seed.seed_everything(999)

Global seed set to 999


In [3]:
import os
if os.path.isdir('ds_utils'):
    !rm -rf ds_utils
!git clone -q https://github.com/respect5716/ds_utils.git

from ds_utils.pytorch_lightning import BaseModule, BaseDataModule, CheckpointCallback, AttributeDict
from ds_utils.metric import accuracy

In [10]:
args = AttributeDict({
    'batch_size': 128,
    'base_dir': '/content/drive/Shared drives/Yoon/Project/Doing/Deep Learning Paper Implementation',

    'trainer': {
        'accelerator': 'dp',
        'gpus': -1,
        'log_every_n_steps': 10,
        'num_sanity_val_steps': -1,
        'max_epochs': 10,
        'check_val_every_n_epoch': 1,
    },

    'optimizer': {
        'name': 'sgd',
        'params': {
            'lr': 1e-2,
            'momentum': 0.9,
            'weight_decay': 5e-4
        }
    },

    'scheduler': {
        'name': 'linear',
        'warmup_size': 0
    }
})

## 2. Data

### Prepare

In [5]:
torchvision.datasets.CIFAR10(root='./data', train=True, download=True)
torchvision.datasets.CIFAR10(root='./data', train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified


Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test

### DataModule

In [6]:
class DataModule(BaseDataModule):
    def __init__(self, batch_size, transform, collate_fn=None):
        super().__init__(batch_size, collate_fn)
        self.transform = transform

    def setup(self, stage: Optional[str] = None):
        self.train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=self.transform['train'])
        self.train_dataset, self.val_dataset = torch.utils.data.random_split(self.train_dataset, [45000, 5000])
        self.test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=self.transform['eval'])
        self.predict_dataset = self.test_dataset

In [7]:
transform = {
    'train': transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2439, 0.2616)),
    ]),
    
    'eval': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2439, 0.2616)),
    ])
}

In [8]:
data_module = DataModule(args.batch_size, transform)
data_module.setup()

Files already downloaded and verified
Files already downloaded and verified


In [9]:
inputs = data_module.sample('predict')
inputs[0].size(), inputs[1].size()

(torch.Size([128, 3, 32, 32]), torch.Size([128]))

## 3. Model

In [11]:
class Block(nn.Module):
    def __init__(self, in_C, out_C):
        super(Block, self).__init__()
        self.conv = nn.Conv2d(in_C, out_C, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(out_C)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

In [19]:
class VGG11(BaseModule):
    def __init__(self, **hparams):
        super().__init__()
        self.save_hyperparameters(hparams)

        self.features = nn.Sequential(
            Block(3, 64),
            nn.MaxPool2d(2),
            Block(64, 128),
            nn.MaxPool2d(2),
            Block(128, 256),
            Block(256, 256),
            nn.MaxPool2d(2),
            Block(256, 512),
            Block(512, 512),
            nn.MaxPool2d(2),
            Block(512, 512),
            Block(512, 512),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.features(x)
        logits = self.classifier(x)
        return logits
    
    def step(self, batch, mode):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = accuracy(logits, y)
        self.log_dict(
            {f'{mode}/loss': loss, f'{mode}/acc': acc}, 
            prog_bar = True, on_step = True, on_epoch = True
        )
        return loss

In [20]:
model = VGG11(**args)

## 4. Experiment

### Train

In [21]:
logger = pl.loggers.WandbLogger(
    project = 'paper',
    log_model = False,
    reinit = True
)

logger.watch(model)

In [22]:
lr_monitor = pl.callbacks.LearningRateMonitor()
ckpt_callback = CheckpointCallback('ckpt', metric='valid/acc_epoch', mode='max')

In [23]:
trainer = pl.Trainer(
    logger = logger,
    callbacks = [lr_monitor, ckpt_callback],
    **args.trainer
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [24]:
trainer.fit(model, data_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.prepare_data has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.prepare_data.
  f"DataModule.{name} has already been called, so it will not be called again. "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | features   | Sequential | 9.2 M 
1 | classifier | Sequential | 5.1 K 
------------------------------------------
9.2 M     Trainable params
0         Non-trainable params
9.2 M     Total params
3

Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 999


STEP: 000000 | valid/acc_epoch: 0.104 | model saved (-10000000000.000 -> 0.104)


Training: -1it [00:00, ?it/s]

...

STEP: 002815 | valid/acc_epoch: 0.805 | model saved (0.768 -> 0.805)


Validating: 0it [00:00, ?it/s]

STEP: 003167 | valid/acc_epoch: 0.812 | model saved (0.805 -> 0.812)


Validating: 0it [00:00, ?it/s]

STEP: 003519 | valid/acc_epoch: 0.819 | model saved (0.812 -> 0.819)


### Test

In [28]:
model = VGG11.load_from_checkpoint('ckpt').freeze()

In [29]:
res = trainer.test(model, data_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.prepare_data has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.prepare_data.
  f"DataModule.{name} has already been called, so it will not be called again. "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test/acc': 0.8241999745368958,
 'test/acc_epoch': 0.8241999745368958,
 'test/loss': 0.5107018351554871,
 'test/loss_epoch': 0.5107018351554871}
--------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  f"DataModule.{name} has already been called, so it will not be called again. "


### Inference